In [ ]:
#!/usr/bin/env python3

import logging
import random
import re
from collections import namedtuple

try: input = raw_input
except NameError: pass

log = logging.getLogger(__name__)


class Key:
    def __init__(self, word, weight, decomps):
        self.word = word
        self.weight = weight
        self.decomps = decomps


class Decomp:
    def __init__(self, parts, save, reasmbs):
        self.parts = parts
        self.save = save
        self.reasmbs = reasmbs
        self.next_reasmb_index = 0


class Eliza:
    def __init__(self):
        self.initials = []
        self.finals = []
        self.quits = []
        self.pres = {}
        self.posts = {}
        self.synons = {}
        self.keys = {}
        self.memory = []

    def load(self, path):
        key = None
        decomp = None
        with open(path) as file:
            for line in file:
                if not line.strip():
                    continue
                tag, content = [part.strip() for part in line.split(':')]
                if tag == 'initial':
                    self.initials.append(content)
                elif tag == 'final':
                    self.finals.append(content)
                elif tag == 'quit':
                    self.quits.append(content)
                elif tag == 'pre':
                    parts = content.split(' ')
                    self.pres[parts[0]] = parts[1:]
                elif tag == 'post':
                    parts = content.split(' ')
                    self.posts[parts[0]] = parts[1:]
                elif tag == 'synon':
                    parts = content.split(' ')
                    self.synons[parts[0]] = parts
                elif tag == 'key':
                    parts = content.split(' ')
                    word = parts[0]
                    weight = int(parts[1]) if len(parts) > 1 else 1
                    key = Key(word, weight, [])
                    self.keys[word] = key
                elif tag == 'decomp':
                    parts = content.split(' ')
                    save = False
                    if parts[0] == '$':
                        save = True
                        parts = parts[1:]
                    decomp = Decomp(parts, save, [])
                    key.decomps.append(decomp)
                elif tag == 'reasmb':
                    parts = content.split(' ')
                    decomp.reasmbs.append(parts)

    def _match_decomp_r(self, parts, words, results):
        if not parts and not words:
            return True
        if not parts or (not words and parts != ['*']):
            return False
        if parts[0] == '*':
            for index in range(len(words), -1, -1):
                results.append(words[:index])
                if self._match_decomp_r(parts[1:], words[index:], results):
                    return True
                results.pop()
            return False
        elif parts[0].startswith('@'):
            root = parts[0][1:]
            if not root in self.synons:
                raise ValueError("Unknown synonym root {}".format(root))
            if not words[0].lower() in self.synons[root]:
                return False
            results.append([words[0]])
            return self._match_decomp_r(parts[1:], words[1:], results)
        elif parts[0].lower() != words[0].lower():
            return False
        else:
            return self._match_decomp_r(parts[1:], words[1:], results)

    def _match_decomp(self, parts, words):
        results = []
        if self._match_decomp_r(parts, words, results):
            return results
        return None

    def _next_reasmb(self, decomp):
        index = decomp.next_reasmb_index
        result = decomp.reasmbs[index % len(decomp.reasmbs)]
        decomp.next_reasmb_index = index + 1
        return result

    def _reassemble(self, reasmb, results):
        output = []
        for reword in reasmb:
            if not reword:
                continue
            if reword[0] == '(' and reword[-1] == ')':
                index = int(reword[1:-1])
                if index < 1 or index > len(results):
                    raise ValueError("Invalid result index {}".format(index))
                insert = results[index - 1]
                for punct in [',', '.', ';']:
                    if punct in insert:
                        insert = insert[:insert.index(punct)]
                output.extend(insert)
            else:
                output.append(reword)
        return output

    def _sub(self, words, sub):
        output = []
        for word in words:
            word_lower = word.lower()
            if word_lower in sub:
                output.extend(sub[word_lower])
            else:
                output.append(word)
        return output

    def _match_key(self, words, key):
        for decomp in key.decomps:
            results = self._match_decomp(decomp.parts, words)
            if results is None:
                print('Decomp did not match: %s', decomp.parts)
                continue
            print('Decomp matched: %s', decomp.parts)
            print('Decomp results: %s', results)
            results = [self._sub(words, self.posts) for words in results]
            print('Decomp results after posts: %s', results)
            reasmb = self._next_reasmb(decomp)
            print('Using reassembly: %s', reasmb)
            if reasmb[0] == 'goto':
                goto_key = reasmb[1]
                if not goto_key in self.keys:
                    raise ValueError("Invalid goto key {}".format(goto_key))
                print('Goto key: %s', goto_key)
                return self._match_key(words, self.keys[goto_key])
            output = self._reassemble(reasmb, results)
            if decomp.save:
                self.memory.append(output)
                print('Saved to memory: %s', output)
                continue
            return output
        return None

    def respond(self, text):
        if text.lower() in self.quits:
            return None

        text = re.sub(r'\s*\.+\s*', ' . ', text)
        text = re.sub(r'\s*,+\s*', ' , ', text)
        text = re.sub(r'\s*;+\s*', ' ; ', text)
        print('After punctuation cleanup: %s', text)

        words = [w for w in text.split(' ') if w]
        print('Input: %s', words)

        words = self._sub(words, self.pres)
        print('After pre-substitution: %s', words)

        keys = [self.keys[w.lower()] for w in words if w.lower() in self.keys]
        keys = sorted(keys, key=lambda k: -k.weight)
        print('Sorted keys: %s', [(k.word, k.weight) for k in keys])

        output = None

        for key in keys:
            output = self._match_key(words, key)
            if output:
                print('Output from key: %s', output)
                break
        if not output:
            if self.memory:
                index = random.randrange(len(self.memory))
                output = self.memory.pop(index)
                print('Output from memory: %s', output)
            else:
                output = self._next_reasmb(self.keys['xnone'].decomps[0])
                print('Output from xnone: %s', output)

        return " ".join(output)

    def initial(self):
        return random.choice(self.initials)

    def final(self):
        return random.choice(self.finals)

    def run(self):
        print(self.initial())

        while True:
            sent = input('> ')

            output = self.respond(sent)
            if output is None:
                break

            print(output)

        print(self.final())


def main():
    eliza = Eliza()
    eliza.load('doctor.txt')
    eliza.run()

if __name__ == '__main__':
    logging.basicConfig(level = logging.DEBUG,
                        filename = 'app.log', 
                        filemode = 'w', 
                        format = '%(name)s - %(levelname)s - %(message)s')
    main()


In [ ]:
Good morning I am an organ dealer. What do you need ?

> Hello mister
After punctuation cleanup: %s Hello mister
Input: %s ['Hello', 'mister']
After pre-substitution: %s ['Hello', 'mister']
Sorted keys: %s [('hello', 1)]
Decomp matched: %s ['*']
Decomp results: %s [['Hello', 'mister']]
Decomp results after posts: %s [['Hello', 'mister']]
Using reassembly: %s ['What', 'do', 'you', 'need', '?']
Output from key: %s ['What', 'do', 'you', 'need', '?']
What do you need ?

> Organ dealer ?
After punctuation cleanup: %s Organ dealer ?
Input: %s ['Organ', 'dealer', '?']
After pre-substitution: %s ['Organ', 'dealer', '?']
Sorted keys: %s []
Output from xnone: %s ['What', 'you', 'tell', 'me', 'does', 'not', 'interest', 'me,', 'I', 'have', 'to', 'go', 'sell', 'a', 'heart', 'and', 'above', 'all', 'do', 'not', 'pay', 'attention', 'to', 'the', 'screams.']
What you tell me does not interest me, I have to go sell a heart and above all do not pay attention to the screams.

> Why you are organ dealer ?
After punctuation cleanup: %s Why you are organ dealer ?
Input: %s ['Why', 'you', 'are', 'organ', 'dealer', '?']
After pre-substitution: %s ['Why', 'you', 'are', 'organ', 'dealer', '?']
Sorted keys: %s [('why', 1)]
Decomp did not match: %s ['*', 'why', 'not', '*']
Output from xnone: %s ["You're", 'not', 'saying', 'anything', 'really', 'valid,', 'do', 'you', 'want', 'an', 'eye', 'to', 'see', 'more', 'clearly', '?']
You're not saying anything really valid, do you want an eye to see more clearly ?

> Sorry bro ..
After punctuation cleanup: %s Sorry bro . 
Input: %s ['Sorry', 'bro', '.']
After pre-substitution: %s ['Sorry', 'bro', '.']
Sorted keys: %s [('sorry', 1)]
Decomp matched: %s ['*']
Decomp results: %s [['Sorry', 'bro', '.']]
Decomp results after posts: %s [['Sorry', 'bro', '.']]
Using reassembly: %s ["Don't", 'apologize,', "let's", 'get', 'back', 'to', 'business.']
Output from key: %s ["Don't", 'apologize,', "let's", 'get', 'back', 'to', 'business.']
Don't apologize, let's get back to business.

> Is legal ?
After punctuation cleanup: %s Is legal ?
Input: %s ['Is', 'legal', '?']
After pre-substitution: %s ['Is', 'legal', '?']
Sorted keys: %s [('legal', 1)]
Decomp matched: %s ['*']
Decomp results: %s [['Is', 'legal', '?']]
Decomp results after posts: %s [['Is', 'legal', '?']]
Using reassembly: %s ['That', 'does', 'not', 'concern', 'you.']
Output from key: %s ['That', 'does', 'not', 'concern', 'you.']
That does not concern you.

> This work is really legal ?
After punctuation cleanup: %s This work is really legal ?
Input: %s ['This', 'work', 'is', 'really', 'legal', '?']
After pre-substitution: %s ['This', 'work', 'is', 'really', 'legal', '?']
Sorted keys: %s [('legal', 1)]
Decomp matched: %s ['*']
Decomp results: %s [['This', 'work', 'is', 'really', 'legal', '?']]
Decomp results after posts: %s [['This', 'work', 'is', 'really', 'legal', '?']]
Using reassembly: %s ['Are', 'you', 'with', 'the', 'FBI', 'or', 'the', 'NSA', '?']
Output from key: %s ['Are', 'you', 'with', 'the', 'FBI', 'or', 'the', 'NSA', '?']
Are you with the FBI or the NSA ?

> Do you sells lungs ?
After punctuation cleanup: %s Do you sells lungs ?
Input: %s ['Do', 'you', 'sells', 'lungs', '?']
After pre-substitution: %s ['Do', 'you', 'sells', 'lungs', '?']
Sorted keys: %s [('sells', 1)]
Decomp matched: %s ['*', 'do', 'you', 'sells', '*']
Decomp results: %s [[], ['lungs', '?']]
Decomp results after posts: %s [[], ['lungs', '?']]
Using reassembly: %s ['Look', 'at', 'the', 'catalog,', "I'm", 'not', 'sure', 'if', 'I', 'sell', 'a', '(2)']
Output from key: %s ['Look', 'at', 'the', 'catalog,', "I'm", 'not', 'sure', 'if', 'I', 'sell', 'a', 'lungs', '?']
Look at the catalog, I'm not sure if I sell a lungs ?

> How much two lungs ?
After punctuation cleanup: %s How much two lungs ?
Input: %s ['How', 'much', 'two', 'lungs', '?']
After pre-substitution: %s ['How', 'much', 'two', 'lungs', '?']
Sorted keys: %s [('how', 1)]
Decomp matched: %s ['*', 'how', 'much', '*']
Decomp results: %s [[], ['two', 'lungs', '?']]
Decomp results after posts: %s [[], ['two', 'lungs', '?']]
Using reassembly: %s ['It', 'depends', 'on', 'the', 'state', 'of', 'the', 'organ.']
Output from key: %s ['It', 'depends', 'on', 'the', 'state', 'of', 'the', 'organ.']
It depends on the state of the organ.

> I buy two lungs     
After punctuation cleanup: %s I buy two lungs
Input: %s ['I', 'buy', 'two', 'lungs']
After pre-substitution: %s ['I', 'buy', 'two', 'lungs']
Sorted keys: %s [('buy', 4)]
Decomp matched: %s ['*', 'i', 'buy', '*']
Decomp results: %s [[], ['two', 'lungs']]
Decomp results after posts: %s [[], ['two', 'lungs']]
Using reassembly: %s ['I', 'can', 'give', 'you', 'a', 'fresh', 'stomach', 'cut', 'this', 'morning', 'with', '(2)', '.']
Output from key: %s ['I', 'can', 'give', 'you', 'a', 'fresh', 'stomach', 'cut', 'this', 'morning', 'with', 'two', 'lungs', '.']
I can give you a fresh stomach cut this morning with two lungs .

> I want another lungs    
After punctuation cleanup: %s I want another lungs
Input: %s ['I', 'want', 'another', 'lungs']
After pre-substitution: %s ['I', 'want', 'another', 'lungs']
Sorted keys: %s [('want', 1)]
Decomp matched: %s ['*', 'i', 'want', '*']
Decomp results: %s [[], ['another', 'lungs']]
Decomp results after posts: %s [[], ['another', 'lungs']]
Using reassembly: %s ['Okay,', "I'll", 'make', 'you', 'a', '(2)', '.']
Output from key: %s ['Okay,', "I'll", 'make', 'you', 'a', 'another', 'lungs', '.']
Okay, I'll make you a another lungs .

> Thank you bro
After punctuation cleanup: %s Thank you bro
Input: %s ['Thank', 'you', 'bro']
After pre-substitution: %s ['Thank', 'you', 'bro']
Sorted keys: %s [('thank', 1)]
Decomp matched: %s ['*']
Decomp results: %s [['Thank', 'you', 'bro']]
Decomp results after posts: %s [['Thank', 'I', 'bro']]
Using reassembly: %s ["You're", 'welcome,', "don't", 'be', 'controlled.']
Output from key: %s ["You're", 'welcome,', "don't", 'be', 'controlled.']
You're welcome, don't be controlled.

> Goodbye
Goodbye. I'm glad I did business with someone so nice. Come back when you want, you will have a promotion on the new organs.

#Exe Eliza (eliza.py)


In [ ]:
Hi! I am a chatbot created by Camille for your service
>Hey
Hey there
>Are you a robot ?
I don't want to brag, but I got a B + on the Turing test
>Let's play rock sheet scissors
Okay, let's go
>rock
rock
>sheet
rock
>scissors
sheet
>What is your name ?
I am a bot created by Camille. you can call me Finme !
>Why Finme ?
Finme is just the name they gave when I got the job. It doesn't mean anything in particular. But I love this.
>Who is the best chatbot ?
Is there any more ?
>Test 1,2,3
I am fully operational and all my circuits are working perfectly
>Do you have a family ?
I don't have a family tree. But I have a very good directory of files.
>I need to buy an organ
Ask Eliza
>What is the meaning of life ?
42.
>What is your favorite movie ?
I've heard that 'Blade Runner' is a very realistic and sensitive portrayal of intelligent assistants.
>quit
It was nice talking to you. See you soon :)

#Exe NLTK (chatbot_nltk.py)